In [1]:
#| default_exp foundation.codeformat

# Code Formatting
> Tests to check code formatting and quality

Using the `setup.cfg` in this project clean up code cells

In [ ]:
#| export
from __future__ import annotations

In [1]:
#| export
import black
import itertools
import re
import subprocess
from tokenize import tokenize,COMMENT
from ast import parse,FunctionDef
from io import BytesIO
from textwrap import dedent
from types import SimpleNamespace
from inspect import getsource,isfunction,isclass,signature,Parameter
from fastcore.basics import *
import ast
import sys

In [2]:
#| export
def unparse(source:str = None):
    "Unparses `source` with the correct unparser"
    if source is None:
        return ''
    elif sys.version_info.major <= 3 and sys.version_info.minor < 9:
        import astunparse
        return astunparse.unparse(source).rstrip()
    else:
        return ast.unparse(source)

In [3]:
#| export
def parse(source:str = None):
    "Shortcut for `ast.parse`"
    return ast.parse(source)

In [4]:
#| export
def get_annotations(
    parsed_function: ast.FunctionDef # A parsed function
):
    "Extracts type annotations from a single function"
    arg_annotations = []
    for annotation in parsed_function.args.args:
        arg_annotations.append(
            unparse(
                annotation.annotation
            )
        )
    return_annotation = unparse(parsed_function.returns)
    return arg_annotations, return_annotation        

In [79]:
function_source = '''def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int,float): # The sum of a and b
    "Adds two numbers together"
    return a+b'''
annotations = get_annotations(ast.parse(function_source).body[0])
assert annotations == (['(int, float)', "int"], '(int, float)')

In [80]:
#| export
def _parses(s):
    """Parse Python code in string or function object `s`"""
    return parse(dedent(getsource(s) if isfunction(s) else s))

In [81]:
#| export
def _tokens(s):
    """Tokenize Python code in string or function object `s`"""
    if isfunction(s): s = getsource(s)
    return tokenize(BytesIO(s.encode('utf-8')).readline)

In [82]:
#| export
_clean_re = re.compile('^\s*#(.*)\s*$')

In [83]:
#| export
def _clean_comment(s):
    res = _clean_re.findall(s)
    return res[0] if res else None

In [84]:
#| export
def _param_locs(s, returns=True):
    """`dict` of parameter line numbers to names"""
    body = _parses(s).body
    if len(body)!= 1 or not isinstance(body[0], FunctionDef): return None
    defn = body[0]
    res = {arg.lineno:arg.arg for arg in defn.args.args}
    if returns and defn.returns: res[defn.returns.lineno] = 'return'
    return res

In [85]:
#| export
def _get_comment(line, arg, comments, parms):
    if line in comments: return comments[line].strip()
    line -= 1
    res = []
    while line and line in comments and line not in parms:
        res.append(comments[line])
        line -= 1
    return dedent('\n'.join(reversed(res))) if res else None

In [86]:
#| export
def _get_full(anno, name, default, docs):
    if anno==Parameter.empty and default!=Parameter.empty: anno = type(default)
    return AttrDict(docment=docs.get(name), anno=anno, default=default)

In [87]:
#| export
def docments(s, full=False, returns=True, eval_str=False):
    """`dict` of parameter names to 'docment-style' comments in function or string `s`"""
    if isclass(s): s = s.__init__ # Constructor for a class
    comments = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
    parms = _param_locs(s, returns=returns)
    docs = {arg:_get_comment(line, arg, comments, parms) for line,arg in parms.items()}
    if not full: return docs

    if isinstance(s,str): s = eval(s)
    sig = signature(s)
    res = {arg:_get_full(p.annotation, p.name, p.default, docs) for arg,p in sig.parameters.items()}
    if returns: res['return'] = _get_full(sig.return_annotation, 'return', Parameter.empty, docs)
    if eval_str:
        hints = type_hints(s)
        for k,v in res.items():
            if k in hints: v['anno'] = hints.get(k)
    return res

In [500]:
#| export
def reformat_function(
    source:str, # Source code
):
    "Refactors `source` function docstring"
    function_docments = docments(source)
    parsed_source = parse(source).body[0]
    function_annotations = get_annotations(parsed_source)
    # Extract the docstring
    if isinstance(parsed_source.body[0], ast.Expr):
        function_docstring = unparse(parsed_source.body[0]).replace("'", "")
        start_line_num = parsed_source.body[0].end_lineno
    else:
        function_docstring = ""
        start_line_num = parsed_source.body[0].lineno - 1
    # Extract the function logic without formatting
    function_body = "\n".join(source.split("\n")[start_line_num:])
    variables = list(function_docments.keys())
    for variable in ("self", "cls", "return"):
        if variable in variables:
            variables.remove(variable)
    if len(variables) > 0 and any(docstring != None for docstring in function_docments.values()):
        if len(function_docstring) > 0:
            function_docstring += "\n\n"
        function_docstring += f"Args:\n"
        for i, name in enumerate(function_docments.keys()):
            if name in ("self", "cls"):
                continue
            elif name == "return":
                arg_doc = function_docments[name]
                if arg_doc != None:
                    annotation = function_annotations[1]
                    function_docstring += "\nReturns:\n"
                    function_docstring += f'    {annotation}'
                    function_docstring += f': {arg_doc}\n'
            else:
                arg_doc = function_docments.get(name, "")
                annotation = function_annotations[0][i]
                if annotation != '':
                    if not annotation.startswith("("):
                        annotation = f'({annotation})'
                function_docstring += f"    {name}"
                if annotation != None:
                    function_docstring += f' {annotation}'
                function_docstring += f":\n        {arg_doc}\n"
    offset = parsed_source.body[0].col_offset
    offset = ' '*offset
    if function_docstring != '':
        function_docstring = '\n'.join([
            f'{offset}{line}'
            for line in function_docstring.split("\n")
        ])
        function_docstring = f'"""\n{function_docstring}'
        if "return" in function_docments:
            function_docstring += '\n'
        if not function_docstring.endswith(offset):
            if not function_docstring.endswith("\n"):
                function_docstring += "\n"
            function_docstring += f'{offset}"""'
        else:
            function_docstring += f'"""'
        function_docstring = parse(function_docstring)
        if not isinstance(parsed_source.body[0], ast.Expr):
            parsed_source.body.insert(0, function_docstring)
        else:
            parsed_source.body[0] = function_docstring
    else:
        parsed_source.body.insert(0, ast.parse(''))
    parsed_source.body = parsed_source.body[:1]
    return f'{unparse(parsed_source)}\n{function_body}'

In [503]:
print(reformat_function(function_source))

def addition(a: (int, float), b: int=2) -> (int, float):
    """
    Adds two numbers together
    
    Args:
        a (int, float):
            The first number to add
        b (int):
            The second number to add
    
    Returns:
        (int, float): The sum of a and b
    """
    return a+b


In [454]:
#| export
def reformat_class(
    source:str # Source code of an entire class
):
    "Reformats an entire class source code"
    parsed_class = parse(source)
    new_source = ""
    
    if isinstance(parsed_class.body[0].body[0], ast.Expr):
        class_docstring = unparse(parsed_class.body[0].body[0]).replace("'", "")
        docstring_offset = f"{' '*parsed_class.body[0].body[0].col_offset}"
        if not class_docstring.startswith("\\n"):
            class_docstring = f'\n{docstring_offset}{class_docstring}'
        if not class_docstring.endswith(f"\\n{docstring_offset}"):
            class_docstring = f'{class_docstring}\n{docstring_offset}'
        class_docstring = ast.parse(f'"""{class_docstring}"""')
    else:
        class_docstring = ""
    for index, item in enumerate(parsed_class.body[0].body[1:]):
        if not isinstance(item, (ast.FunctionDef, ast.ClassDef)):
            # Comments in the source code
            original_source = source.split("\n")
            comment = "\n".join(original_source[item.lineno-1:item.end_lineno])
            row = 1
            if len(original_source) > item.lineno-1+row:
                while original_source[item.lineno-1+row].lstrip().startswith("#"):
                    comment += "\n"
                    comment += "\n".join(original_source[item.lineno-1+row:item.lineno-1+row+1])
                    row += 1
            original_source = comment
            if not original_source[0].isspace():
                refactored = "\n".join([
                    f'{" "*(item.col_offset)}{line}'
                    for line in original_source.split("\n")
                ])
            else:
                refactored = original_source
            new_source += f'\n{refactored}'
        else:
            original_source = '\n'.join(
                source.split('\n')[item.lineno-1:item.end_lineno]
            )
            if original_source[0].isspace():
                original_source = "\n".join([
                    line[item.col_offset:]
                    for line in original_source.split("\n")
                ])
            if isinstance(item, ast.FunctionDef):
                # Assumes only one decorator for now
                has_decorator = False
                for decorator in item.decorator_list:
                    decorator = source.split('\n')[item.lineno-2:item.lineno-1]
                    decorator = '\n'.join([
                        line[item.col_offset:] 
                        for line in decorator
                    ])
                    has_decorator = True
                refactored = reformat_function(original_source)
                if has_decorator:
                    refactored = f'{decorator}\n{refactored}'
                refactored = refactored.split('\n')
                refactored = '\n'.join([
                    f'{" "*item.col_offset}{line}'
                    for line in refactored
                ])
                new_source += f'\n\n{refactored}'
            elif isinstance(item, ast.ClassDef):
                # De-indent the inner class
                refactored = reformat_class(original_source).split("\n")
                if '"' in refactored[1] and refactored[2] == '':
                    refactored.pop(2)
                refactored = '\n'.join([
                    f'{" "*item.col_offset}{line}'
                    for line in refactored
                ])
                new_source += f'\n\n{refactored}'
    
    parsed_class.body[0].body = parsed_class.body[0].body[:1]
    if not isinstance(parsed_class.body[0].body[0], ast.Expr):
        parsed_class.body[0].body.insert(0, class_docstring)
    else:
        parsed_class.body[0].body[0] = class_docstring
    return f'{unparse(parsed_class)}{new_source}'

In [455]:
class_source = '''class StrEnum(str, Enum):
    """
    Same as the `StrEnum` available in Python 3.11, returns
    the lower-cased version of the member name.
    """
    @classmethod
    def contains(cls, value) -> bool:
        """
        Checks if `value` exists in the current `Enum`
        """
        return value in cls._value2member_map_
    
    def _generate_next_value_(name, start, count, last_values):
        return name.lower()'''

In [456]:
print(reformat_class(class_source))

class StrEnum(str, Enum):
    """
    Same as the `StrEnum` available in Python 3.11, returns
    the lower-cased version of the member name.
    """

    @classmethod
    def contains(cls, value) -> bool:
        """
        
        Checks if `value` exists in the current `Enum`
        
        """
        return value in cls._value2member_map_

    def _generate_next_value_(name, start, count, last_values):
        return name.lower()
        return name.lower()


In [457]:
class_source = '''class Arithmetic:
    "A class that can perform basic arithmetic on ops"
    _o = 2
    # Here's a comment
    _b = 5
    _c = 3
    
    class A:
        "My docstring"
        _d = 5
        def __init__(
          self, 
          o:int # An integer
        ):
            self.o = o
    
    def __init__(
        self,
        a:int, # The first number to use
        b:(int, float), # The second number to use
    ):
        self.a = a
        self.b = b
        
    # Here's another test
    
    @delegates()
    def add(
        self
    ) -> (int,float): # Sum of a and b
        "Adds self.a and self.b"
        return self.a + self.b'''

In [458]:
print(reformat_class(class_source))

class Arithmetic:
    """
    A class that can perform basic arithmetic on ops
    """
    _o = 2
    # Here's a comment
    _b = 5
    _c = 3

    class A:
        """
        My docstring
        """
        _d = 5
    
        def __init__(self, o: int):
            """
            Args:
                o (int):
                    An integer
            """
            self.o = o

    def __init__(self, a: int, b: (int, float)):
        """
        Args:
            a (int):
                The first number to use
            b (int, float):
                The second number to use
        """
        self.a = a
        self.b = b

    @delegates()
    def add(self) -> (int, float):
        """
        Adds self.a and self.b
        """
        return self.a + self.b


In [222]:
example = """attach_example(
  source_code = "some code",
  example_code = "my example"
)"""

In [223]:
#| export
def attach_example(
    source_code:str, # The complete python code of a function or class
    example_code:str # The complete annotated example to go along with the source code
) -> str:
    "Attaches example code to source code in a code cell"
    beginning, docstring, end = source_code.split('\"\"\"')
    search = re.search("^(\s*)(?:\S|$)", docstring.split("\n")[0].lstrip("\n"))
    if search is None:
        num_spaces = 4
    else:
        num_spaces = len(search.groups()[0]) + 4
    spaces = " " * num_spaces
    if docstring.endswith(f"\n{spaces}"):
        docstring = docstring[:(-1 - num_spaces)]
    docstring += f"{spaces}Example:\n\n{spaces}```python\n"
    example_code = example_code.split("\n")
    for i,line in enumerate(example_code):
        if line[0].isspace() or line[0] in [")", "}"]:
            example_code[i] = f'{spaces}... {line}'
        else:
            example_code[i] = f'{spaces}>>> {line}'
    example_code += [f"{spaces}```\n{spaces}"]
    docstring += '\n'.join(example_code)
    return '\"\"\"'.join((beginning, docstring, end))

In [ ]:
#| example
attach_example(
  source_code = "some code",
  example_code = "my example"
)

In [227]:
print(attach_example(reformat_function(function_source), example))

def addition(a: (int, float), b: int=2) -> (int, float):
    """
    Adds two numbers together
    
    Args:
        a (int, float):
            The first number to add
        b (int):
            The second number to add
    
    Returns:
        (int, float): The sum of a and b
    
    Example:

    ```python
    >>> attach_example(
    ...   source_code = "some code",
    ...   example_code = "my example"
    ... )
    ```
    """
    return a+b


In [228]:
#| export
from nbdev.processors import Processor
from nbdev.process import extract_directives

In [236]:
#| export
class CodeCleaner(Processor):
    def cell(self, cell):
        if cell.cell_type == "code" and "export" in cell.directives_ and "import" not in cell.source:
            if "class " in cell.source:
                cell.source = reformat_class(cell.source)
            elif "def " in cell.source:
                cell.source = reformat_function(cell.source)

In [237]:
#| export
class DocstringExampleAdder(Processor):
    has_example = False
    previous_cell = None
    def cell(self, cell):
        if cell.cell_type == "code":
            if "export" in cell.directives_:
                if "import" not in cell.source and ("def " in cell.source or "class " in cell.source):
                    self.previous_cell = cell
            if "example" in cell.directives_:
                self.previous_cell.source = attach_example(
                    self.previous_cell.source, 
                    cell.source
                )
                self.previous_cell = None
                self.nb.cells.remove(cell)

In [238]:
from nbdev.processors import mk_cell, NBProcessor, dict2nb

In [239]:
nb = {
    "cells":[
    mk_cell(f'#| export\n{function_source}', "code"),
    mk_cell("#| example\n" + example, "code"),
]}

In [240]:
processor = NBProcessor(procs=[CodeCleaner, DocstringExampleAdder], nb=dict2nb(nb))

In [241]:
processor.process()

In [242]:
print(processor.nb["cells"][0].source)

def addition(a: (int, float), b: int=2) -> (int, float):
    """
    Adds two numbers together
    
    Args:
        a (int, float):
            The first number to add
        b (int):
            The second number to add
    
    Returns:
        (int, float): The sum of a and b
    
    Example:

    ```python
    >>> attach_example(
    ...   source_code = "some code",
    ...   example_code = "my example"
    ... )
    ```
    """
    return a+b
